In [19]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [44]:
# The paper where the idea came out well to have multiple layers of neurons
# https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [ ]:
list_chars = sorted(list(set(''.join(words))))
stoi = {}
itos = {}
idx = 1
for ch in list_chars:
    stoi[ch] = idx
    itos[idx] = ch
    idx += 1

stoi['.'] = 0
itos[0] = '.'

itos

In [114]:
block_size = 3
X = []
Y = []
for w in words:
    context = [0] * block_size
    # print(w)
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context) + "--->" + ch)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)
# YU = Y.unsqueeze(1)
# Z = torch.cat((X, YU), dim=1)

X.shape, X.dtype, Y.shape, Y.dtype


(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [51]:
C = torch.randn(27, 2)
C

tensor([[ 1.4473, -1.2141],
        [-0.9716,  1.2108],
        [ 0.0863,  0.9163],
        [-1.0872,  0.2015],
        [-0.6956, -1.5113],
        [ 0.6718,  0.5016],
        [-0.5835,  0.2084],
        [ 1.2900,  1.4108],
        [-0.4708, -1.1117],
        [-0.8985,  1.1597],
        [ 0.7848,  1.0986],
        [-1.9655, -0.3301],
        [-0.3415,  2.6753],
        [ 0.7178, -0.9203],
        [-1.3833, -0.0451],
        [ 1.6798,  0.6419],
        [ 0.0185,  0.4023],
        [ 0.1917,  0.0652],
        [-0.2936, -0.0505],
        [-0.0259, -1.4352],
        [-0.5894, -1.6930],
        [-0.2674, -1.7367],
        [ 2.0828,  0.8604],
        [ 2.0151, -0.0510],
        [ 1.6528,  1.5525],
        [ 1.7770,  0.3908],
        [-1.0825,  0.2686]])

In [48]:
C[5]

tensor([ 0.6071, -1.5800])

In [54]:
C[torch.tensor([5,6,7,7,7,7,7,7])]

tensor([[ 0.6718,  0.5016],
        [-0.5835,  0.2084],
        [ 1.2900,  1.4108],
        [ 1.2900,  1.4108],
        [ 1.2900,  1.4108],
        [ 1.2900,  1.4108],
        [ 1.2900,  1.4108],
        [ 1.2900,  1.4108]])

In [57]:
C[X].shape

torch.Size([32, 3, 2])

In [58]:
W1 = torch.randn(6, 100)
b1 = torch.randn(100)

In [59]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [60]:
emb @ W1 + b1

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [67]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

In [69]:
torch.cat(torch.unbind(emb, dim=1), 1)

tensor([[ 1.4473, -1.2141,  1.4473, -1.2141,  1.4473, -1.2141],
        [ 1.4473, -1.2141,  1.4473, -1.2141,  0.6718,  0.5016],
        [ 1.4473, -1.2141,  0.6718,  0.5016,  0.7178, -0.9203],
        [ 0.6718,  0.5016,  0.7178, -0.9203,  0.7178, -0.9203],
        [ 0.7178, -0.9203,  0.7178, -0.9203, -0.9716,  1.2108],
        [ 1.4473, -1.2141,  1.4473, -1.2141,  1.4473, -1.2141],
        [ 1.4473, -1.2141,  1.4473, -1.2141,  1.6798,  0.6419],
        [ 1.4473, -1.2141,  1.6798,  0.6419, -0.3415,  2.6753],
        [ 1.6798,  0.6419, -0.3415,  2.6753, -0.8985,  1.1597],
        [-0.3415,  2.6753, -0.8985,  1.1597,  2.0828,  0.8604],
        [-0.8985,  1.1597,  2.0828,  0.8604, -0.8985,  1.1597],
        [ 2.0828,  0.8604, -0.8985,  1.1597, -0.9716,  1.2108],
        [ 1.4473, -1.2141,  1.4473, -1.2141,  1.4473, -1.2141],
        [ 1.4473, -1.2141,  1.4473, -1.2141, -0.9716,  1.2108],
        [ 1.4473, -1.2141, -0.9716,  1.2108,  2.0828,  0.8604],
        [-0.9716,  1.2108,  2.0828,  0.8

In [71]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [ ]:
a.view(2,9)

In [ ]:
a.storage()

In [ ]:
emb.view(32, 6)

In [80]:
h = emb.view(-1, 6) @ W1 + b1
h = torch.tanh(h)

In [82]:
h.shape

torch.Size([32, 100])

In [83]:
W2 = torch.randn(100, 27)
b2 = torch.randn(27)

In [85]:
logits = h @ W2 + b2

In [86]:
logits.shape

torch.Size([32, 27])

In [87]:
counts = logits.exp()

In [88]:
probs = counts / counts.sum(1, keepdim=True)

In [89]:
probs.shape

torch.Size([32, 27])

In [91]:
probs[0].sum()

tensor(1.0000)

In [92]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [93]:
probs[torch.arange(32), Y]

tensor([1.0471e-14, 9.1520e-08, 1.2989e-08, 5.8865e-05, 1.6654e-13, 9.2179e-20,
        8.7875e-12, 7.0675e-09, 1.9360e-05, 2.4143e-15, 8.2993e-23, 1.0039e-13,
        4.1100e-07, 6.5672e-05, 3.9982e-17, 4.6292e-12, 1.7289e-12, 1.2912e-02,
        3.8837e-09, 5.4563e-03, 1.7709e-09, 1.0413e-05, 1.3930e-08, 7.0765e-21,
        6.8238e-09, 6.1220e-07, 6.9836e-14, 2.3298e-01, 2.6497e-05, 6.3187e-07,
        7.8651e-14, 2.4912e-15])

In [94]:
-probs[torch.arange(32), Y].log().mean()

tensor(22.2764)

In [ ]:
## - Cleaning and re-writing ##

In [209]:
def build_dataset(words):
    X = []
    Y = []
    block_size = 3
    context = [0] * block_size

    for w in words:
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(18)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182600, 3]) torch.Size([182600])
torch.Size([22780, 3]) torch.Size([22780])
torch.Size([22766, 3]) torch.Size([22766])


In [210]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn(27, 2, generator=g)
W1 = torch.randn(6, 100, generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn(100, 27, generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [198]:
sum(p.nelement() for p in parameters)

3481

In [211]:
for p in parameters:
    p.requires_grad = True

In [212]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [ ]:
for _ in range(10):

    #minibatch
    ix = torch.randint(0, X.shape[0], (32,))
    # forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdim=True)
    # loss = -probs[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Y[ix])
    print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.1 * p.grad

In [228]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.2435, grad_fn=<NllLossBackward0>)

In [230]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.2497, grad_fn=<NllLossBackward0>)

In [233]:
# sample names
g = torch.Generator().manual_seed(2147483647 + 10)
for i in range(10):
    context = [0] * block_size
    out = []
    while True:
        emb = C[torch.tensor(context)]
        h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        out.append(ix)
        context = context[1:] + [ix]
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

carmah.
quiri.
tifkimli.
quity.
hacalene.
arhuel.
amerahc.
taquiu.
tamara.
ceriav.
